In [1]:
import nltk
import itertools
import operator
import pandas as pd
import numpy as np
nltk.download('vader_lexicon')

[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

C:\Users\deept\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [3]:
reviews = pd.read_csv('yelp__review.csv', low_memory=False)

In [4]:
reviews[0:2]

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,0,dDl8zu1vWPdKGihJrwQbpw,msQe1u7Z_XuqjGoqhB0J5g,pomGBqfbxcqPv14c3XH-ZQ,5.0,2012-11-13,I love this place! My fiance And I go here atl...,0.0,0.0,0.0
1,0,LZp4UX5zK3e-c5ZGSeo3kA,msQe1u7Z_XuqjGoqhB0J5g,jtQARsP6P-LbkyjbO1qNGg,1.0,2014-10-23,Terrible. Dry corn bread. Rib tips were all fa...,3.0,1.0,1.0


In [ ]:
reviews[0:1]['text']

In [5]:
reviews_df = reviews[['business_id','text']]

In [ ]:
reviews_df.head()

In [6]:
reviews_df.count()

business_id    5996997
text           5996994
dtype: int64

In [7]:
sentiment_analyzer = SentimentIntensityAnalyzer()

In [37]:
reviews_df.groupby('business_id')
reviews_df.nunique()

business_id     188595
text           5992993
dtype: int64

It can be observed that there are just **188595** business ids while **5992993** texts, which implies the dataset consists of multiple reviews given by different users for the same business. Review text has to now be grouped based on the business id

In [8]:
df = reviews_df.groupby('business_id')['text'].nunique()
print(df)

business_id
--1UhMGODdWsrMastO9DZw      24
--6MefnULPED_I942VcFNA      39
--7zmmkVg-IMGaXbuVd0SQ      54
--8LPVSo5i0Oo61X01sV9A       4
--9QQLMTbFzLJ_oT-ON3Xw      11
--9e1ONYQuAa-CB_Rrw7Tw    1545
--DaPTJW3-tB1vP-PfdTEg      45
--DdmeR16TRb3LsjG0ejrQ       5
--EF5N7P70J_UYBTPypYlA       5
--EX4rRznJrltyn-34Jz1w       4
--FBCX-N37CMYDfs790Bnw     125
--FLdgM0GNpXVMn74ppCGw      12
--GM_ORV2cYS-h38DSaCLw       8
--Gc998IMjLn8yr-HTzGUg       3
--I7YYLada0tSLkORTHb5Q      83
--KCl2FvVQpvjzmZSPyviA      13
--KQsXc-clkO7oHRqGzSzg      35
--Ni3oJ4VOqfOEu7Sj2Vzg       6
--Rsj71PBe31h5YljVseKA       8
--S62v0QgkqQaVUhFnNHrw      35
--SrzpvFLwP_YFwB_Cetow      44
--TcDRzRIxhvHM4DSgEuMA       5
--U98MNlDym2cLn36BBPgQ       4
--VMPfs4zfZJtQbqzJsNhg      10
--WsruI0IGEoeRmkErU5Gg      15
--Y7NhBKzLTbNliMUX_wfg      12
--YPwqIlRJrhHkJcjY3eiA       3
--ab39IjZR_xUf81WyTyHg      11
--cZ6Hhc9F7VkKXxHMVZSQ     333
--cgVkbWTiga3OYTkymKqA      13
                          ... 
zzXuJF6UUT1kgTyCsmgDmQ     

In [9]:
# dict_ = reviews_df.set_index('business_id')['text'].to_dict()
dict_ = {k: list(v) for k,v in reviews_df.groupby('business_id')['text']}

Once data is grouped, it is of the form **business_id:[review1, review2, ..., reviewN]**. Find the sentiment of each of the reviews and store it as a **dict[business_id]: [sent1, sent2, sent3..., sentN]**.

In [10]:
count = 0
upper_limit = 0
lower_limit = 0
sent = {}
while count!=170000:
    lower_limit=upper_limit
    upper_limit+=10000
    x = itertools.islice(dict_.items(), lower_limit,upper_limit)
    for key,val in x:
        count += 1
        classify = []
#         if key not in sent:
#             sent[key] = []
        try:
            for val_item in val:
                sentiment = sentiment_analyzer.polarity_scores(val_item)
                sentiment.pop('compound')
                sentiment.pop('neu')
                classify.append(max(sentiment.items(), key=operator.itemgetter(1))[0])
            sent[key] = classify
        except:
            continue
    print(lower_limit, upper_limit, count)

0 10000 10000
10000 20000 20000
20000 30000 30000
30000 40000 40000
40000 50000 50000
50000 60000 60000
60000 70000 70000
70000 80000 80000
80000 90000 90000
90000 100000 100000
100000 110000 110000
110000 120000 120000
120000 130000 130000
130000 140000 140000
140000 150000 150000
150000 160000 160000
160000 170000 170000


In [11]:
x = itertools.islice(dict_.items(),170001,188595)
for key,val in x:
    count += 1
    classify = []
#         if key not in sent:
#             sent[key] = []
    try:
        for val_item in val:
            sentiment = sentiment_analyzer.polarity_scores(val_item)
            sentiment.pop('compound')
            sentiment.pop('neu')
            classify.append(max(sentiment.items(), key=operator.itemgetter(1))[0])
        sent[key] = classify
    except:
        continue

Find the max occurence of the sentiment, that determines if the overall feedback is pos or neg

In [27]:
final_class = {}
for k,v in sent.items():
#     print(k,v)
    pos_neg = {'pos':0, 'neg':0}
    for val in v:
#         print(val)
        if val == 'pos':
            pos_neg['pos']+=1
        else:
            pos_neg['neg']+=1
#     print(pos_neg)
    final_class[k] = max(pos_neg.items(), key=operator.itemgetter(1))[0]

**Business_id and collabarative sentiment of the review texts:**

In [28]:
final_class

{'--1UhMGODdWsrMastO9DZw': 'pos',
 '--6MefnULPED_I942VcFNA': 'pos',
 '--7zmmkVg-IMGaXbuVd0SQ': 'pos',
 '--8LPVSo5i0Oo61X01sV9A': 'pos',
 '--9QQLMTbFzLJ_oT-ON3Xw': 'pos',
 '--9e1ONYQuAa-CB_Rrw7Tw': 'pos',
 '--DaPTJW3-tB1vP-PfdTEg': 'pos',
 '--DdmeR16TRb3LsjG0ejrQ': 'pos',
 '--EF5N7P70J_UYBTPypYlA': 'pos',
 '--EX4rRznJrltyn-34Jz1w': 'pos',
 '--FBCX-N37CMYDfs790Bnw': 'pos',
 '--FLdgM0GNpXVMn74ppCGw': 'pos',
 '--GM_ORV2cYS-h38DSaCLw': 'pos',
 '--Gc998IMjLn8yr-HTzGUg': 'neg',
 '--I7YYLada0tSLkORTHb5Q': 'pos',
 '--KCl2FvVQpvjzmZSPyviA': 'pos',
 '--KQsXc-clkO7oHRqGzSzg': 'pos',
 '--Ni3oJ4VOqfOEu7Sj2Vzg': 'neg',
 '--Rsj71PBe31h5YljVseKA': 'pos',
 '--S62v0QgkqQaVUhFnNHrw': 'pos',
 '--SrzpvFLwP_YFwB_Cetow': 'pos',
 '--TcDRzRIxhvHM4DSgEuMA': 'neg',
 '--U98MNlDym2cLn36BBPgQ': 'pos',
 '--VMPfs4zfZJtQbqzJsNhg': 'neg',
 '--WsruI0IGEoeRmkErU5Gg': 'pos',
 '--Y7NhBKzLTbNliMUX_wfg': 'pos',
 '--YPwqIlRJrhHkJcjY3eiA': 'pos',
 '--ab39IjZR_xUf81WyTyHg': 'pos',
 '--cZ6Hhc9F7VkKXxHMVZSQ': 'pos',
 '--cgVkbWTiga

**Final count of positive and negative ratings:**

In [34]:
sum(1 for x in final_class.values() if x == 'pos')

172476

In [35]:
sum(1 for x in final_class.values() if x == 'neg')

16115